In [39]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor



In [17]:
df = pd.read_csv(r'D:\2025 lessons\AI+ML course amaliyot\Datasets\seoul_bike_sharing_demand\SeoulBikeData.csv', encoding='cp949')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      8760 non-null   object 
 1   Rented Bike Count         8760 non-null   int64  
 2   Hour                      8760 non-null   int64  
 3   Temperature(캜)            8760 non-null   float64
 4   Humidity(%)               8760 non-null   int64  
 5   Wind speed (m/s)          8760 non-null   float64
 6   Visibility (10m)          8760 non-null   int64  
 7   Dew point temperature(캜)  8760 non-null   float64
 8   Solar Radiation (MJ/m2)   8760 non-null   float64
 9   Rainfall(mm)              8760 non-null   float64
 10  Snowfall (cm)             8760 non-null   float64
 11  Seasons                   8760 non-null   object 
 12  Holiday                   8760 non-null   object 
 13  Functioning Day           8760 non-null   object 
dtypes: float

In [19]:
df.head()

,Date,Rented Bike Count,Hour,Temperature(캜),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(캜),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [20]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

In [21]:
df =df.drop(columns=['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Rented Bike Count         8760 non-null   int64  
 1   Hour                      8760 non-null   int64  
 2   Temperature(캜)            8760 non-null   float64
 3   Humidity(%)               8760 non-null   int64  
 4   Wind speed (m/s)          8760 non-null   float64
 5   Visibility (10m)          8760 non-null   int64  
 6   Dew point temperature(캜)  8760 non-null   float64
 7   Solar Radiation (MJ/m2)   8760 non-null   float64
 8   Rainfall(mm)              8760 non-null   float64
 9   Snowfall (cm)             8760 non-null   float64
 10  Seasons                   8760 non-null   object 
 11  Holiday                   8760 non-null   object 
 12  Functioning Day           8760 non-null   object 
 13  Year                      8760 non-null   int32  
 14  Month   

In [22]:
df['Holiday'].value_counts()

Holiday
No Holiday    8328
Holiday        432
Name: count, dtype: int64

In [29]:
from datacleaner import autoclean
df= autoclean(df)

c:\Users\user\.virtualenvs\AI+ML_course_amaliyot-8lQVJ-6X\Lib\site-packages\datacleaner\datacleaner.py:77: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  input_dataframe[column].fillna(input_dataframe[column].median(), inplace=True)
c:\Users\user\.virtualenvs\AI+ML_course_amaliyot-8lQVJ-6X\Lib\site-packages\datacleaner\datacleaner.py:77: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method wil

In [59]:
df.head()

,Rented Bike Count,Hour,Temperature(캜),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(캜),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,Year,Month,Day
0,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,3,1,1,2017,12,1
1,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,3,1,1,2017,12,1
2,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,3,1,1,2017,12,1
3,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,3,1,1,2017,12,1
4,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,3,1,1,2017,12,1


In [ ]:
x = df.drop(columns=['Rented Bike Count'])
y = df['Rented Bke Count']
x_train,x_temp, y_train, y_temp =  train_test_split(x, y, test_size=0.2, random_state=42)
x_test , x_val , y_test, y_val = train_test_split(x_temp,y_temp, test_size=0.5, random_state=42)


In [35]:
x_test.shape

(876, 15)

In [34]:
x_val.shape

(876, 15)

# XGBoost Model

In [43]:
xgb_model = XGBRegressor().fit(x_train,y_train)

In [66]:
xgb_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [44]:
xgb_y_pred = xgb_model.predict(x_test)

In [48]:
xgb_r2 = r2_score(y_test, xgb_y_pred)
xgb_mse = mean_squared_error(y_test,xgb_y_pred)


# LightGB Model

In [49]:
lgb_model = LGBMRegressor().fit(x_train,y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000321 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1299
[LightGBM] [Info] Number of data points in the train set: 7008, number of used features: 15
[LightGBM] [Info] Start training from score 704.767837


In [65]:
lgb_model

LGBMRegressor()

In [ ]:
lgbm_y_pred = lgb_model.predict(x_test)

In [75]:
lgbm_r2 = r2_score(y_test, lgbm_y_pred)
lgbm_mse = mean_squared_error(y_test,lgbm_y_pred)

In [76]:
lgbm_r2

-0.7167064837653949

In [77]:
lgbm_mse

775385.7581006298

# Gradient Boosting Model

In [63]:
gboost_model = GradientBoostingRegressor().fit(x_train,y_train)

In [64]:
gboost_model

GradientBoostingRegressor()

In [68]:
gboost_y_pred = gboost_model.predict(x_test)

In [78]:
gboost_r2 = r2_score(y_test, gboost_y_pred)
gboost_mse = mean_squared_error(y_test,gboost_y_pred)
gboost_mse

76313.71400954376

In [79]:
from tabulate import tabulate
header = ['Metrics' , 'XGBRegressor', 'LGBMRegressor', 'GradientBoostingRegressor']
table = ['R2_score', xgb_r2, lgbm_r2, gboost_r2], ['MSE', xgb_mse, lgbm_mse ,gboost_mse]

print(tabulate(table,header, tablefmt='grid'))

+-----------+----------------+-----------------+-----------------------------+
| Metrics   |   XGBRegressor |   LGBMRegressor |   GradientBoostingRegressor |
+===========+================+=================+=============================+
| R2_score  |       0.886386 |       -0.716706 |                    0.831041 |
+-----------+----------------+-----------------+-----------------------------+
| MSE       |   51316.1      |   775386        |                76313.7      |
+-----------+----------------+-----------------+-----------------------------+
